# Exploring Sports Dataset through a Dask Dataframe

   *The link to this dataset:*
    https://academictorrents.com/details/2a81590d3b32e6ddd8a87f1ec4f08205098476ee

### As we got to know more about gpx files, we moved on to putting the data in a dask dataframe form.

#### Since this is a very large dataset, we decided to choose five folders (each containing the data for one athlete), and from each folder, 100 gpx files(each containing all points of one journey).
#### First, we needed these files as .csv files, so we opened the gpx files and saved the data as relevant lists in corresponding csv files, adding a few desired fields for our future columns.
#### Now our files are ready for creating a dask dataframe

In [261]:
%%time
import os
import gpxpy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from os.path import join
from os import listdir
import csv


base_dir = 'sport/'

for athlete_dir in listdir(base_dir):
    count = 1
    for athlete_file in listdir(join(base_dir, athlete_dir)):
        if '.csv' in athlete_file:
            continue
        if count > 100:   #We just collect 100 files for each athlete, due to ram limitation
            break
        gpx_file = open(join(base_dir, athlete_dir, athlete_file), 'r')
        gpx = gpxpy.parse(gpx_file)
        file_list = []
        for point in gpx.tracks[0].segments[0].points:
            file_list.append([
                point.longitude,
                point.latitude,
                point.elevation,
                point.time,
                athlete_dir,
                athlete_file.replace('.gpx', ''),
                0 # We will put the distance between the current point and the previous one, in this column
            ])
        name = join(base_dir, athlete_dir, athlete_file.replace('.gpx','.csv'))
        with open(name, 'w', newline='') as myfile:
            wr = csv.writer(myfile)
            wr.writerows(file_list)
        count += 1


Wall time: 5min 45s


In [262]:
%%time
import dask
import dask.dataframe as dd

sport = dd.read_csv('sport\*\*.csv')

Wall time: 2.54 s


In [263]:
sport

,16.173866288736463,46.538401767611504,132.39999389648438,2014-04-07 12:57:11+00:00,Athlete1,1,0
npartitions=500,,,,,,,
,float64,float64,float64,object,object,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


#### This is the general look of our dataframe. Now we should change the column names:

In [264]:
columns=['longitude', 'latitude', 'altitude', 'time', 'athlete', 'journey', 'distance']
sport.columns = columns

In [265]:
sport.head()

,longitude,latitude,altitude,time,athlete,journey,distance
0,16.173844,46.538406,132.399994,2014-04-07 12:57:12+00:00,Athlete1,1,0
1,16.173710,46.538436,132.399994,2014-04-07 12:57:16+00:00,Athlete1,1,0
2,16.173465,46.538491,132.399994,2014-04-07 12:57:21+00:00,Athlete1,1,0
3,16.173376,46.538510,132.399994,2014-04-07 12:57:22+00:00,Athlete1,1,0
4,16.173286,46.538527,132.399994,2014-04-07 12:57:23+00:00,Athlete1,1,0


In [266]:
sport.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, longitude to distance
dtypes: object(2), float64(3), int64(2)

#### Looks like dask dataframe info() only gives a brief explanation about the data.
#### But we can see that columns athlete and time are in string format. We will change the time column to datetime format:

In [267]:
sport['time'] = dd.to_datetime(sport['time'])

#### Now, we want to fill our distance column. For each journey, the distance field will contain the distance between the current point and the previous form. We will do this by the following function (this time we used geopy library):

In [268]:
from geopy import distance

def calculate_distance(pdf):
    prev_point = None
    for i, point in pdf.iterrows():
        if prev_point is None:
            prev_point = point
            continue
        if prev_point.journey != point.journey:
            prev_point = point
            continue
        lon1 = prev_point.longitude
        lon2 = point.longitude
        lat1 = prev_point.latitude
        lat2 = point.latitude
        pdf.at[i, 'distance'] = distance.distance((lon1, lat1), (lon2, lat2)).m
        prev_point = point
    return pdf

#### For adding the distance values to our dataframe, we should pass the function to the map_partitions method of our dask dataframe, so it would be applied to the chunks of data without a problem:

In [269]:
%%time
sport_with_distances = sport.map_partitions(calculate_distance).compute()
sport_with_distances.head(100)

Wall time: 22min 6s


,longitude,latitude,altitude,time,athlete,journey,distance
0,16.173844,46.538406,132.399994,2014-04-07 12:57:12+00:00,Athlete1,1,0
1,16.173710,46.538436,132.399994,2014-04-07 12:57:16+00:00,Athlete1,1,15
2,16.173465,46.538491,132.399994,2014-04-07 12:57:21+00:00,Athlete1,1,27
3,16.173376,46.538510,132.399994,2014-04-07 12:57:22+00:00,Athlete1,1,10
4,16.173286,46.538527,132.399994,2014-04-07 12:57:23+00:00,Athlete1,1,10
...,...,...,...,...,...,...,...
95,16.148912,46.539183,133.800003,2014-04-07 13:01:25+00:00,Athlete1,1,17
96,16.148970,46.539027,133.800003,2014-04-07 13:01:27+00:00,Athlete1,1,17
97,16.148993,46.538947,133.800003,2014-04-07 13:01:28+00:00,Athlete1,1,8
98,16.149011,46.538782,133.800003,2014-04-07 13:01:30+00:00,Athlete1,1,17


### Now we can analyse our dataset. 

#### First let's see how many points in total is recorded for each athlete:

In [270]:
%%time
sport_with_distances.athlete.value_counts()

Wall time: 161 ms


Athlete3    455812
Athlete1    402747
Athlete2    237991
Athlete5    208650
Athlete4     95637
Name: athlete, dtype: int64

#### Then we can see the number of points per journey:

In [271]:
%%time
sport_with_distances.groupby(['athlete']).journey.value_counts()

Wall time: 249 ms


athlete   journey
Athlete1  124        16234
          128        14917
          132        14787
          137        14315
          12         14287
                     ...  
Athlete5  1060          91
          1046          89
          1055          71
          105            1
          1004           1
Name: journey, Length: 500, dtype: int64

#### Here is some information about the distance of journies:

In [272]:
journey_distance = sport_with_distances.groupby(['athlete', 'journey']).distance.sum()


In [273]:
journey_distance

athlete   journey
Athlete1  1           53686
          10          77777
          11          67102
          12         135414
          13             13
                      ...  
Athlete5  1084        75644
          1085        11176
          1086        32664
          1087        33425
          1088        40276
Name: distance, Length: 500, dtype: int64

In [274]:
jd = pd.DataFrame(journey_distance['Athlete1'][10:40])
jd.reset_index(inplace=True)
jd.journey = jd.journey.astype(str)
jd.distance /= 1000

In [276]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import Viridis256
from bokeh.transform import factor_cmap

In [277]:
import math
source = ColumnDataSource(jd)
journies = source.data['journey'].tolist()
p = figure(x_range=journies)
color_map = factor_cmap(field_name='journey',
                    palette=Viridis256[:30], factors=journies)

p.vbar(x='journey', top='distance', source=source, width=0.70, color=color_map)

p.title.text ='Sample distance of 30 journies for Athlete1'
p.xaxis.axis_label = '30 journies of Athlete1'
p.yaxis.axis_label = 'journey distance(KM)'
p.xaxis.major_label_orientation = math.pi/2

In [278]:
show(p)

In [279]:
mean_dist = pd.DataFrame(journey_distance.groupby('athlete').mean())
mean_dist.reset_index(inplace=True)
mean_dist.distance /= 1000

In [280]:
source = ColumnDataSource(mean_dist)
athletes = source.data['athlete'].tolist()
p = figure(x_range=athletes, plot_width=500, plot_height=400)
color_map = factor_cmap(field_name='athlete',
                    palette=['#f03b20']*5, factors=athletes)

p.vbar(x='athlete', top='distance', source=source, width=0.50, color=color_map)

p.title.text ='Comparing Average distance for each athlete'
p.xaxis.axis_label = 'Athletes'
p.yaxis.axis_label = 'Average distance of all journies(KM)'
show(p)

#### By calculating the average values per journey of each athlete, we get an average point (average longitude, latitude and altitude), and the average distance between each two consecutive points:

In [281]:
%%time
sport_with_distances.groupby(['athlete', 'journey']).mean()

Wall time: 252 ms


longitude   latitude    altitude   distance
athlete  journey                                             
Athlete1 1        16.089062  46.551292  149.621018  29.693584
         10       16.238507  46.604686  249.252825  36.618173
         11       16.208636  46.727600  187.640630  33.006394
         12       16.148202  46.655455  201.949969   9.478127
         13       16.175591  46.538365  213.567850   0.008100
...                     ...        ...         ...        ...
Athlete5 1084     16.079646  46.707787  229.460922  41.517014
         1085     16.135989  46.623897  189.508622   7.587237
         1086     16.102885  46.648603  201.778076  18.750861
         1087     16.165949  46.616286  464.653480  40.811966
         1088     16.212161  46.588399  198.049738   9.155717

[500 rows x 4 columns]

#### From the standard deviations, it can be infered that the longitudes and latitudes of all journey points are close to the mean value, while for some journies, the standard deviation for altitudes is high which means that the journey had a wider range of different elevations, maybe cycling over a hill or something like that:

In [282]:
%%time
sport_with_distances.groupby(['athlete', 'journey']).std()

Wall time: 254 ms


longitude  latitude    altitude   distance
athlete  journey                                            
Athlete1 1         0.061710  0.019868   10.926711  24.400553
         10        0.055719  0.057992    7.743303  28.182246
         11        0.046910  0.091368   40.648476  34.189411
         12        0.072500  0.082505   46.113463   4.481806
         13        0.000004  0.000009    1.040867   0.159673
...                     ...       ...         ...        ...
Athlete5 1084      0.034170  0.058849   48.693994  26.526467
         1085      0.008670  0.005763    2.998168   1.669955
         1086      0.032566  0.017609    4.283261   5.806663
         1087      0.013253  0.022053  185.473329  22.746048
         1088      0.034997  0.017568    4.084833   3.167103

[500 rows x 4 columns]

#### Then we get the difference between the minimum and maximum values for each journey:

In [283]:
%%time
min_df = sport_with_distances.groupby(['athlete', 'journey']).min()
min_df.head()

Wall time: 260 ms


longitude   latitude    altitude                      time  \
athlete  journey                                                               
Athlete1 1        16.006548  46.518628  131.600006 2014-04-07 12:57:12+00:00   
         10       16.164186  46.509978  238.399994 2015-02-23 11:07:42+00:00   
         11       16.161779  46.538263  129.000000 2015-06-10 14:14:09+00:00   
         12       16.019659  46.509934  159.199997 2015-06-30 12:39:25+00:00   
         13       16.175580  46.538262  211.600006 2016-02-23 14:58:36+00:00   

                  distance  
athlete  journey            
Athlete1 1               0  
         10              0  
         11              0  
         12              0  
         13              0

In [284]:
%%time
max_df = sport_with_distances.groupby(['athlete', 'journey']).max()
max_df.head()

Wall time: 232 ms


longitude   latitude    altitude                      time  \
athlete  journey                                                               
Athlete1 1        16.194670  46.595207  175.600006 2014-04-07 14:22:58+00:00   
         10       16.345301  46.687761  266.399994 2015-02-23 13:16:23+00:00   
         11       16.321814  46.824616  277.200012 2015-06-10 16:13:29+00:00   
         12       16.289549  46.798497  351.000000 2015-06-30 17:57:26+00:00   
         13       16.175642  46.538383  215.800003 2016-02-23 15:25:30+00:00   

                  distance  
athlete  journey            
Athlete1 1             101  
         10            268  
         11           1012  
         12            394  
         13              4

In [285]:
%%time
diff = max_df - min_df
diff.head()

Wall time: 16 ms


longitude  latitude    altitude     time  distance
athlete  journey                                                    
Athlete1 1         0.188122  0.076579   44.000000 01:25:46       101
         10        0.181114  0.177783   28.000000 02:08:41       268
         11        0.160035  0.286353  148.200012 01:59:20      1012
         12        0.269890  0.288563  191.800003 05:18:01       394
         13        0.000061  0.000121    4.199997 00:26:54         4

#### We can get some useful information from diff. The altitude column shows the difference between the highest and lowst point of the journey. and we have the total duration of the journey in time column. So we can calculate the average speed of the athlete.

#### First let's see the average duration of journeys for each athlete:

In [286]:
%%time
sum_duration = diff.groupby(['athlete']).time.sum()
sum_duration

Wall time: 5 ms


athlete
Athlete1   10 days 04:43:10
Athlete2    8 days 06:09:22
Athlete3    7 days 06:59:20
Athlete4    6 days 18:28:05
Athlete5    5 days 04:51:21
Name: time, dtype: timedelta64[ns]

In [287]:
mean_duration = sum_duration/100
mean_duration

athlete
Athlete1   02:26:49.900000
Athlete2   01:58:53.620000
Athlete3   01:44:59.600000
Athlete4   01:37:28.850000
Athlete5   01:14:54.810000
Name: time, dtype: timedelta64[ns]

#### Since we collected 100 journies per athlete, the average duration can be calculated by dividing the total duration of all journies to 100.

#### For calculating the speed, first we need to calculate time durations in hours, in float format:

In [288]:
%%time
for i, journey in diff.iterrows():
    diff.at[i, 'hours'] = journey.time.seconds/3600
diff 

Wall time: 112 ms


longitude  latitude    altitude     time  distance     hours
athlete  journey                                                              
Athlete1 1         0.188122  0.076579   44.000000 01:25:46       101  1.429444
         10        0.181114  0.177783   28.000000 02:08:41       268  2.144722
         11        0.160035  0.286353  148.200012 01:59:20      1012  1.988889
         12        0.269890  0.288563  191.800003 05:18:01       394  5.300278
         13        0.000061  0.000121    4.199997 00:26:54         4  0.448333
...                     ...       ...         ...      ...       ...       ...
Athlete5 1084      0.132868  0.182205  173.199982 02:51:22       100  2.856111
         1085      0.029091  0.019612   21.399994 00:50:01        19  0.833611
         1086      0.105480  0.064251   15.400009 00:55:27        48  0.924167
         1087      0.051129  0.080459  487.000000 00:18:41       188  0.311389
         1088      0.110702  0.059893   15.399994 01:25:17        19  1.421389

[500 rows x 6 columns]

#### Speed in Km/H:

In [289]:
%%time
speed = journey_distance/(1000*(diff.hours))
speed.head()

Wall time: 2 ms


athlete   journey
Athlete1  1          37.557248
          10         36.264370
          11         33.738436
          12         25.548472
          13          0.028996
dtype: float64

In [290]:
speed.std()

19.93517230645446

#### From the standard deviation, we can see that all the speeds were somewhat close to the average.

#### In this plot, we can see the comparison between the average speed of the athletes:

In [291]:
speed_df = pd.DataFrame(speed.groupby('athlete').mean())
speed_df.reset_index(inplace=True)
speed_df.columns=['athlete', 'speed']
source = ColumnDataSource(speed_df)
athletes = source.data['athlete'].tolist()
p = figure(x_range=athletes, plot_width=500, plot_height=400)
color_map = factor_cmap(field_name='athlete',
                    palette=['#756bb1']*5, factors=athletes)

p.vbar(x='athlete', top='speed', source=source, width=0.50, color=color_map)

p.title.text ='Comparing average speed for each athlete'
p.xaxis.axis_label = 'Athletes'
p.yaxis.axis_label = 'Average speed'
show(p)

#### let's see if the height difference of journies is correlated to the speed:

In [292]:
diff['speed'] = speed
diff['altitude'].corr(diff['speed'])

0.22825142641796312

#### There is almost no correlations

#### We chose two journeys to see how the change of altitude was in each:

In [293]:
sample_journey = sport_with_distances[sport_with_distances['athlete']=='Athlete5']
sample_journey = sample_journey[sample_journey['journey']==1087]
sample_journey['time'] = pd.Series([val.time() for val in sample_journey['time']])

In [294]:
sample_journey2 = sport_with_distances[sport_with_distances['athlete']=='Athlete1']
sample_journey2 = sample_journey2[sample_journey2['journey']==10]
sample_journey2['time'] = pd.Series([val.time() for val in sample_journey2['time']])

In [295]:
source = ColumnDataSource(sample_journey)
source2 = ColumnDataSource(sample_journey2)



p = figure(x_axis_type='datetime', plot_height=400, plot_width=900)

p.line(x='time', y='altitude', line_width=2, source=source, color='orchid', legend_label='Athlete5')
p.line(x='time', y='altitude', line_width=2, source=source2, color='olive', legend_label='Athlete1')

p.xaxis.axis_label = 'Athlete 5: journey 1087 and Athlete1: journey 10 time span'
p.yaxis.axis_label = 'altitude'

show(p)

#### We can clearly see that the first journey had much higher height difference. Now we see the second plot in a smaller altitude range:

In [296]:

p = figure(x_axis_type='datetime', plot_height=400)

p.line(x='time', y='altitude', line_width=2, source=source2, color='olive', legend_label='Athlete1')


p.xaxis.axis_label = 'Athlete 1, journey number 10 time span'
p.yaxis.axis_label = 'altitude'

show(p)